In [142]:
import pandas as pd
import numpy as np

In [149]:
data = pd.read_csv('merged.csv')
data

/var/folders/4l/zq4qzqyn6y11sl62hvdnc41r0000gn/T/ipykernel_10736/3524259394.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('merged.csv')


,subject_id,hourly_bin,temp,WBC,HeartRate,RR
0,3,2101-10-21 02:00:00,36.200001,NaN,NaN,18.0
1,3,2101-10-21 03:00:00,36.500000,NaN,NaN,18.0
2,3,2101-10-21 07:00:00,36.900002,NaN,97.0,18.0
3,3,2101-10-21 14:00:00,37.099998,NaN,NaN,18.0
4,3,2101-10-21 16:00:00,37.400002,NaN,NaN,18.0
...,...,...,...,...,...,...
1062112,32810,2100-09-19 10:00:00,NaN,NaN,NaN,30.0
1062113,32810,2100-09-19 12:00:00,NaN,NaN,NaN,28.0
1062114,32810,2100-09-19 13:00:00,NaN,NaN,NaN,30.0
1062115,32810,2100-09-19 14:00:00,NaN,NaN,NaN,30.0


In [150]:
def get_mean_std(data, col):
    std = data[col].describe().loc['std']
    mn = data[col].describe().loc['mean']
    return mn, std

In [151]:
def fill_na(data, col):
    total_null = data[col].isna().sum()
    mn, std = get_mean_std(data, col)
    mean_vary = np.random.choice(np.arange(mn-(std*2), mn + (std*2) + 1), total_null)
    null_indices = data[col].isna()
    data.loc[null_indices, col] = mean_vary
    return data

In [152]:
cols = ['temp', 'HeartRate', 'RR']
for col in cols:
    data = fill_na(data, col)
    
data.WBC.replace(to_replace=['<0.1*', 'no data', '.', 'ERROR', '<0.1'], value=np.nan, inplace=True)
data.WBC = data.WBC.apply(float)
data['WBC'] = data.groupby('subject_id').WBC.fillna(method = 'ffill')
data['WBC'] = data.WBC.fillna(data.WBC.mean())
data

,subject_id,hourly_bin,temp,WBC,HeartRate,RR
0,3,2101-10-21 02:00:00,36.200001,12.230747,143.803169,18.0
1,3,2101-10-21 03:00:00,36.500000,12.230747,134.803169,18.0
2,3,2101-10-21 07:00:00,36.900002,12.230747,97.000000,18.0
3,3,2101-10-21 14:00:00,37.099998,12.230747,104.803169,18.0
4,3,2101-10-21 16:00:00,37.400002,12.230747,75.803169,18.0
...,...,...,...,...,...,...
1062112,32810,2100-09-19 10:00:00,32.468945,13.900000,135.803169,30.0
1062113,32810,2100-09-19 12:00:00,33.468945,13.900000,143.803169,28.0
1062114,32810,2100-09-19 13:00:00,39.468945,13.900000,41.803169,30.0
1062115,32810,2100-09-19 14:00:00,34.468945,13.900000,138.803169,30.0
